In [1]:
# Email Tuesday, May 28, 2019 at 4:50 PM

import pandas as pd
import numpy as np
import datetime
from haversine import haversine
import json

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))


In [3]:
store_numbers=['49','1243','1817','5340','5354']

store_list_SFTP=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190501-135143-036.txt",sep="|",dtype=str)
store_list_SFTP=store_list_SFTP[store_list_SFTP['location_id'].isin(store_numbers)]
store_list_SFTP=store_list_SFTP[['location_id','zip_cd','latitude_meas','longitude_meas','address_line_1','address_line_2','city_nm','state_nm']].rename(columns={"zip_cd":"store_zip"})
store_list_SFTP['store_zip']=store_list_SFTP['store_zip'].apply(lambda x: x.split("-")[0])
store_list_SFTP['latitude_meas']=store_list_SFTP['latitude_meas'].astype(float)
store_list_SFTP['longitude_meas']=store_list_SFTP['longitude_meas'].astype(float)


In [4]:
store_list_SFTP

,location_id,store_zip,latitude_meas,longitude_meas,address_line_1,address_line_2,city_nm,state_nm
157,49,45150,39.178574,-84.281631,825 MAIN ST,NaN,MILFORD,OH
327,1243,47001,39.075299,-84.894435,888 GREEN BLVD,NaN,AURORA,IN
627,1817,45246,39.285708,-84.464548,11372 PRINCETON PIKE,NaN,CINCINNATI,OH
1293,5340,45248,39.145030,-84.626936,3640 WERK RD.,NaN,CINCINNATI,OH
1299,5354,45236,39.227755,-84.396230,4154 HUNT RD,NaN,BLUE ASH,OH


In [5]:
df_store_lables_zip=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                                  dtype=str,usecols=['zip','location_id','revenue_flag'])
output_1=pd.merge(store_list_SFTP,df_store_lables_zip,on="location_id",how="left")
output_1=output_1[output_1['revenue_flag'].isin(['P','S'])]
print(output_1.shape)
print(output_1['location_id'].nunique())


(53, 10)
3


In [6]:
output_1=output_1.reset_index()
del output_1['index']
output_1['dist_miles']=np.nan
for i,row in output_1.iterrows():
    store_center=(row['latitude_meas'],row['longitude_meas'])
    zip_cd = row['zip']
    
    try:
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
    except:
        print(zip_cd)
    output_1.loc[i,'dist_miles']=dist
    
output_1=output_1[(output_1['revenue_flag']=="P") | (output_1['dist_miles']<=20)]


In [7]:
output_2=store_list_SFTP[~store_list_SFTP['location_id'].isin(output_1['location_id'].tolist())]
output_2=output_2.reset_index()
del output_2['index']
df_store_new_zip=pd.DataFrame(columns=["location_id",'zip','revenue_flag','dist_miles'])

for i,row in output_2.iterrows():
    location_id=row['location_id']
    store_center=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":location_id,"zip":zip_cd,"revenue_flag":"zips_in_10","dist_miles":dist},index=[location_id])
            df_store_new_zip=df_store_new_zip.append(df)
            
df_store_new_zip=df_store_new_zip.reset_index()
del df_store_new_zip['index']       
output_2=pd.merge(output_2,df_store_new_zip,on="location_id",how="left")        

In [8]:
output=output_1.append(output_2)
output=output[output_1.columns.tolist()]

In [9]:
df_unique_zips=output.sort_values(['revenue_flag','dist_miles'])
df_unique_zips=df_unique_zips[['zip','revenue_flag']].drop_duplicates("zip")

In [10]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Zips_5_life_moments_test_Faith_20190529/BL_zips_for_5_life_moments_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zips.to_excel(writer,"unique_zips",index=False)
output.to_excel(writer,"zips_by_store",index=False)
writer.save()